In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#url 내에서 상세 정보 크롤링 해보기
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
browser = webdriver.Chrome(options=options)
browser.maximize_window()

txt = pd.read_csv('./LCK_2019_Spring_Promotion_match_links.txt', header=None)
url_list = [i for i in txt.iloc[:,0]]
url = url_list[0]

browser.get(url)

In [3]:
#경기일
date = url[-10:]

date

'2018-09-22'

In [4]:
#사이드창 닫기
left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
if left_close_button.get_attribute('innerText') == '닫기':
    left_close_button.click()
    
right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
if right_close_button.get_attribute('innerText') == '닫기':
    right_close_button.click()

In [5]:
#매치 스코어
time.sleep(1)
match_score = browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div/div/div[2]').get_attribute('innerText').split('\n')
match_score = match_score[0] + '(' + match_score[1] + ')' + ' : ' + match_score[3] + '(' + match_score[2] + ')'

match_score 

'MVP(0) : BtC(3)'

In [6]:
#경기별 영상 url
video_dict = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}}  #최대 5세트
orders = []
num_set = 3
try:
    videos = browser.find_element(By.CLASS_NAME, 'mb-4').find_elements(By.TAG_NAME, 'li')
    for i in range(len(videos)):
        if i > 4:  #5번째 영상부터는 옆으로 가기 버튼 필요
            next_button = browser.find_element(By.XPATH, '//*[@class="relative"]/div/div[2]')
            next_button.click()
            
        video = videos[i]
        video.click()
        time.sleep(1)

        #영상이 순서대로 올려져 있지 않은 경우도 있어, 영상 frame에서 title을 찾아 몇 번째 게임인지 확인
        browser.switch_to.frame(browser.find_element(By.TAG_NAME, 'iframe'))
        
        try:
            title = browser.find_element(By.TAG_NAME, 'title').get_attribute('innerText')
            game_num = title[title.index('Game') + 5]
            orders.append(int(game_num))
            browser.switch_to.default_content()
            
            soup = BeautifulSoup(browser.page_source, 'lxml')
            video_url = soup.find('iframe')['src']
            
            video_dict[int(game_num)][video.text] = video_url 
        except:
            try:
                browser.switch_to.default_content()
                soup = BeautifulSoup(browser.page_source, 'lxml')
                video_url = soup.find('iframe')['src']
                for order in orders:  #[2, 1, 3]
                    video_dict[order][video.text] = video_url
            except:
                pass
            browser.switch_to.default_content()
                    
            close_button = browser.find_element(By.XPATH, '//*[@class="relative"]/div[2]/button')
            close_button.click()
    
except:  #영상이 아예 없는 경우
    video_dict = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}}

video_dict

{1: {'Full Game': 'https://www.youtube.com/embed/pNrgoPvpQuQ'},
 2: {'Full Game': 'https://www.youtube.com/embed/0Lc-1SvSPI4'},
 3: {'Full Game': 'https://www.youtube.com/embed/DJjf0vdpA9c'},
 4: {},
 5: {}}

In [7]:
#세트 선택
set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  

set_1 = set_list[1]  #2세트 선택
set_1.click()
time.sleep(1)

In [8]:
set_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]').get_attribute('innerText').split('\n')

#경기 시간
set_time = set_data[0]

#팀명
team1_name = set_data[1].upper()
team2_name = set_data[4].upper()

set_time, team1_name, team2_name

('38:50', 'MVP', 'BTC')

In [9]:
#밴(보통 5개이지만, 그렇지 않은 경우도 있음)
team1_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[1]/ul[2]')
team1_banned = team1_banned.find_elements(By.TAG_NAME, 'a')
team1_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team1_banned]

team2_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[1]/ul[2]')
team2_banned = team2_banned.find_elements(By.TAG_NAME, 'a')
team2_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team2_banned]

team1_banned = [banned_champ[0].get_attribute('src') for banned_champ in team1_banned]
team2_banned = [banned_champ[0].get_attribute('src') for banned_champ in team2_banned]

team1_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team1_banned]
team2_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team2_banned]

team1_banned, team2_banned

(['Olaf', 'Jax', 'Shen', 'Varus', 'Camille'],
 ['Urgot', 'Sion', 'Skarner', 'Gragas', 'Sejuani'])

In [10]:
#획득 골드
team1_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[2]').get_attribute('innerText')
team2_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[3]').get_attribute('innerText')

team1_gold, team2_gold

('66,092', '76,215')

In [11]:
#오브젝트(타워, 억제기, 전령, 드래곤, 장로드래곤, 바론)
team1_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[2]')
team1_object = team1_object.text.split(sep='\n')[1:]

team2_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[2]')
team2_object = team2_object.text.split(sep='\n')[1:]

team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron = [team1_object[i] for i in range(6)]
team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron = [team2_object[i] for i in range(6)]

print((team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron))
print((team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron))

('2', '0', '1', '1', '0', '0')
('11', '4', '0', '4', '1', '2')


In [12]:
#KDA(kill/death/assist)
team1_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[1]/span[2]').get_attribute('innerText')
team2_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[3]/span[2]').get_attribute('innerText')

team1_k, team1_d, team1_a = [team1_kda.split(' / ')[i] for i in range(3)]
team2_k, team2_d, team2_a = [team2_kda.split(' / ')[i] for i in range(3)]

print((team1_k, team1_d, team1_a))
print((team2_k, team2_d, team2_a))

('5', '6', '15')
('6', '5', '15')


In [13]:
#진영(blue/red)
data = team1_side = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\t\n')
team1_side = data[:len(data)//2 ][0].split('\n')[2][:-5]
team2_side = data[len(data)//2:][0].split('\n')[2][:-5]

team1_side, team2_side

('Blue', 'Red')

In [14]:
#승패 여부(victory/defeated)
xpath = "//strong[contains(text(), 'Victory')]/.."   #승리팀 찾기
xpath = browser.find_element(By.XPATH, xpath)
victory_team = xpath.get_attribute('innerText')
            
if team1_name == victory_team.split('\n')[0]:
    team1_v_d, team2_v_d = 'Victory', 'Defeat'
else:
    team1_v_d, team2_v_d = 'Defeat', 'Victory'
    
team1_v_d, team2_v_d

('Defeat', 'Victory')

In [15]:
#패치 버전
xpath = "//div[contains(text(), 'Ver.')]"
version = browser.find_element(By.XPATH, xpath).get_attribute('innerText')[5:]
version

'8.17'

In [16]:
browser.quit()

In [17]:
data = []

data.append([date, version, match_score, 1, team1_name, team1_v_d, team1_side, team1_banned, team1_gold, 
             team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron, team1_k, team1_d, team1_a, video_dict[1]])
data.append([date, version, match_score, 1, team2_name, team2_v_d, team2_side, team2_banned, team2_gold, 
             team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron, team2_k, team2_d, team2_a, video_dict[1]])

columns = ['date', 'version', 'match_score', 'set', 'team', 'win/lose', 'side', 'ban', 'gold', 
           'tower','turret', 'herald', 'dragon', 'elderly_dragon', 'baron',
           'k', 'd', 'a', 'videos']
pd.DataFrame(data, columns=columns).set_index(['date', 'match_score', 'set', 'version',  'team'])

win/lose  side  \
date       match_score     set version team                  
2018-09-22 MVP(0) : BtC(3) 1   8.17    MVP    Defeat  Blue   
                                       BTC   Victory   Red   

                                                                                 ban  \
date       match_score     set version team                                            
2018-09-22 MVP(0) : BtC(3) 1   8.17    MVP         [Olaf, Jax, Shen, Varus, Camille]   
                                       BTC   [Urgot, Sion, Skarner, Gragas, Sejuani]   

                                               gold tower turret herald  \
date       match_score     set version team                               
2018-09-22 MVP(0) : BtC(3) 1   8.17    MVP   66,092     2      0      1   
                                       BTC   76,215    11      4      0   

                                            dragon elderly_dragon baron  k  d  \
date       match_score     set version team                                     
2018-09-22 MVP(0) : BtC(3) 1   8.17    MVP       1              0     0  5  6   
                                       BTC       4              1     2  6  5   

                                              a  \
date       match_score     set version team       
2018-09-22 MVP(0) : BtC(3) 1   8.17    MVP   15   
                                       BTC   15   

                                                                                        videos  
date       match_score     set version team                                                     
2018-09-22 MVP(0) : BtC(3) 1   8.17    MVP   {'Full Game': 'https://www.youtube.com/embed/p...  
                                       BTC   {'Full Game': 'https://www.youtube.com/embed/p...

In [2]:
#현재 디렉토리 내 txt 파일 모두 불러오기(리그, 연도별)
def get_txt_files(path, league, year):
    txt_list = [file for file in os.listdir(path) if (file.endswith('txt') and file.startswith(f'{league}_{year}'))]
    return txt_list

def get_urls(txt):
    try:
        url_list = [i for i in pd.read_csv(txt, header=None).iloc[:,0]]
        return url_list
    except:  #빈 txt 파일인 경우
        return [None]

In [3]:
#txt 파일 내 모든 url 크롤링하기
def crawling_team_data(txt_list): 
    from tqdm import tqdm
    final_data = []
    
    total_time = 0
    
    for txt in txt_list:    
        url_list = get_urls(txt)
        start_time = time.time()   #소요시간 체크
        for url_idx, url in enumerate(tqdm(url_list, desc=txt)):        
            try:   
                options = webdriver.ChromeOptions()
                options.headless = True
                options.add_argument('window_size=1920x1080')
                options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
                browser = webdriver.Chrome(options=options)
                browser.maximize_window()
                browser.get(url)
                
                time.sleep(1)
                
                #사이드창 닫기
                left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
                if left_close_button.get_attribute('innerText') == '닫기':
                    left_close_button.click()
                    
                right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
                if right_close_button.get_attribute('innerText') == '닫기':
                    right_close_button.click()

                #대회 정보
                tournament_name = txt[:-16]
                
                #경기일
                date = url[-10:]    
                
                #경기별 영상 url
                video_dict = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}}
                orders = []
                num_set = len(browser.find_elements(By.CLASS_NAME, 'shrink-0'))
                try:
                    videos = browser.find_element(By.CLASS_NAME, 'mb-4').find_elements(By.TAG_NAME, 'li')
                    for i in range(len(videos)):
                        if i > 4:  #5번째 영상부터는 옆으로 가기 버튼 필요
                            next_button = browser.find_element(By.XPATH, '//*[@class="relative"]/div/div[2]')
                            next_button.click()
                            
                        video = videos[i]
                        video.click()
                        time.sleep(1)

                        #영상이 순서대로 올려져 있지 않은 경우도 있어, 영상 frame에서 title을 찾아 몇 번째 게임인지 확인
                        browser.switch_to.frame(browser.find_element(By.TAG_NAME, 'iframe'))
                        
                        try:
                            title = browser.find_element(By.TAG_NAME, 'title').get_attribute('innerText')
                            game_num = title[title.index('Game') + 5]
                            orders.append(int(game_num))
                            browser.switch_to.default_content()
                            
                            soup = BeautifulSoup(browser.page_source, 'lxml')
                            video_url = soup.find('iframe')['src']
                            
                            video_dict[int(game_num)][video.text] = video_url 
                        except:
                            try:
                                browser.switch_to.default_content()
                                soup = BeautifulSoup(browser.page_source, 'lxml')
                                video_url = soup.find('iframe')['src']
                                for order in orders:  #[2, 1, 3]
                                    video_dict[order][video.text] = video_url
                            except:
                                pass
                            browser.switch_to.default_content()
                                    
                            close_button = browser.find_element(By.XPATH, '//*[@class="relative"]/div[2]/button')
                            close_button.click()
                    
                except:  #영상이 아예 없는 경우
                    pass
    
                #패치 버전
                xpath = "//div[contains(text(), 'Ver.')]"
                version = browser.find_element(By.XPATH, xpath).get_attribute('innerText')[5:]
                
                #매치 스코어
                match_score = browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div/div/div[2]').get_attribute('innerText').split('\n')
                match_score = match_score[0] + '(' + match_score[1] + ')' + ' : ' + match_score[3] + '(' + match_score[2] + ')'
                
                #세트 선택
                set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  
                for set_num in range(len(set_list)):
                    current_set = set_list[set_num]
                    current_set.click()
                    
                    time.sleep(1)
                    set_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]').get_attribute('innerText').split('\n')
                    
                    #팀명
                    team1_name = set_data[1]
                    team2_name = set_data[4]
                    
                    #경기 시간
                    set_time = set_data[0]
                    
                    #진영(blue/red)
                    data = team1_side = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\t\n')
                    team1_side = data[:len(data)//2 ][0].split('\n')[2][:-5]
                    team2_side = data[len(data)//2:][0].split('\n')[2][:-5]
                    
                            
                    #밴
                    team1_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[1]/ul[2]')
                    team1_banned = team1_banned.find_elements(By.TAG_NAME, 'a')
                    team1_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team1_banned]

                    team2_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[1]/ul[2]')
                    team2_banned = team2_banned.find_elements(By.TAG_NAME, 'a')
                    team2_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team2_banned]

                    team1_banned = [banned_champ[0].get_attribute('src') for banned_champ in team1_banned]
                    team2_banned = [banned_champ[0].get_attribute('src') for banned_champ in team2_banned]

                    team1_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team1_banned]
                    team2_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team2_banned]
                    
                    #획득 골드
                    team1_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[2]').get_attribute('innerText')
                    team2_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[3]').get_attribute('innerText')
                    
                    #오브젝트(타워, 억제기, 전령, 드래곤, 장로드래곤, 바론)
                    team1_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[2]')
                    team1_object = team1_object.text.split(sep='\n')[1:]

                    team2_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[2]')
                    team2_object = team2_object.text.split(sep='\n')[1:]

                    team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron = [team1_object[i] for i in range(6)]
                    team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron = [team2_object[i] for i in range(6)]
                    
                    #KDA(kill/death/assist)
                    team1_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[1]/span[2]').get_attribute('innerText')
                    team2_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[3]/span[2]').get_attribute('innerText')

                    team1_k, team1_d, team1_a = [team1_kda.split(' / ')[i] for i in range(3)]
                    team2_k, team2_d, team2_a = [team2_kda.split(' / ')[i] for i in range(3)]
                    
                    #승패 여부(victory/defeated)
                    xpath = "//strong[contains(text(), 'Victory')]/.."   #승리팀 찾기
                    xpath = browser.find_element(By.XPATH, xpath)
                    victory_team = xpath.get_attribute('innerText')
                    
                    if team1_name == victory_team.split('\n')[0]:
                        team1_v_d, team2_v_d = 'Victory', 'Defeat'
                    else:
                        team1_v_d, team2_v_d = 'Defeat', 'Victory'

                    final_data.append([tournament_name, date, version, match_score, set_num+1, set_time, 
                                       team1_name, team1_v_d, team1_side, team1_banned, team1_gold, 
                                       team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, 
                                       team1_baron, team1_k, team1_d, team1_a, video_dict[set_num+1]])
                    final_data.append([tournament_name, date, version, match_score, set_num+1, set_time, 
                                       team2_name, team2_v_d, team2_side, team2_banned, team2_gold, 
                                       team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, 
                                       team2_baron, team2_k, team2_d, team2_a, video_dict[set_num+1]])
                
            except:
                pass    
            browser.close()    
        end_time = time.time()
        crawling_time = int(end_time - start_time)
        total_time += crawling_time   
            
    print(f'총 소요 시간 : {round(total_time / 60, 2)}분')    
    print('-'*100)  
    
    columns = ['tournament name', 'date', 'version', 'match score', 'set', 'time', 
               'team', 'win/lose', 'side', 'ban', 'gold', 
               'tower','turret', 'herald', 'dragon', 'elderly dragon', 'baron',
               'k', 'd', 'a', 'video url']
    index = ['tournament name', 'date', 'version', 'match score', 'set', 'time']  #추후 read_csv할 때 사용
    
    df = pd.DataFrame(final_data, columns=columns).set_index(index)    

    return df        

In [4]:
#함수 테스트
txt_list = get_txt_files('./', 'LCK', '2022')   
print(f'LCK txt 파일 개수 : {len(txt_list)}')

txt_list

LCK txt 파일 개수 : 3


['LCK_2022_Spring_Playoffs_match_links.txt',
 'LCK_2022_Spring_Regular season_match_links.txt',
 'LCK_2022_Summer_Regular Season_match_links.txt']

In [21]:
df = crawling_team_data(txt_list[:1])  #우선 txt파일 하나만 사용해보기

print(df.shape)
df

LCK_2022_Spring_Playoffs_match_links.txt: 100%|██████████| 5/5 [02:10<00:00, 26.04s/it]


총 소요 시간 : 2.17분
----------------------------------------------------------------------------------------------------
(38, 15)


team  \
tournament name          date       version match score     set time         
LCK_2022_Spring_Playoffs 2022-04-02 12.5    T1(3) : GEN(1)  1   30:09  GEN   
                                                                30:09   T1   
                                                            2   35:38  GEN   
                                                                35:38   T1   
                                                            3   37:27   T1   
                                                                37:27  GEN   
                                                            4   27:55  GEN   
                                                                27:55   T1   
                         2022-03-27 12.5    GEN(3) : DK(2)  1   38:20  GEN   
                                                                38:20   DK   
                                                            2   37:27   DK   
                                                                37:27  GEN   
                                                            3   35:20  GEN   
                                                                35:20   DK   
                                                            4   33:03  GEN   
                                                                33:03   DK   
                                                            5   34:06   DK   
                                                                34:06  GEN   
                         2022-03-26 12.5    T1(3) : KDF(0)  1   26:17   T1   
                                                                26:17  KDF   
                                                            2   20:16  KDF   
                                                                20:16   T1   
                         2022-03-24 12.5    DRX(2) : KDF(3) 1   37:27  DRX   
                                                                37:27  KDF   
                                                            2   40:17  DRX   
                                                                40:17  KDF   
                                                            3   35:10  KDF   
                                                                35:10  DRX   
                                                            4   30:34  DRX   
                                                                30:34  KDF   
                                                            5   40:02  KDF   
                                                                40:02  DRX   
                         2022-03-23 12.5    DK(3) : BRO(0)  1   39:54   DK   
                                                                39:54  BRO   
                                                            2   32:35  BRO   
                                                                32:35   DK   
                                                            3   26:55   DK   
                                                                26:55  BRO   

                                                                      win/lose  \
tournament name          date       version match score     set time             
LCK_2022_Spring_Playoffs 2022-04-02 12.5    T1(3) : GEN(1)  1   30:09   Defeat   
                                                                30:09  Victory   
                                                            2   35:38  Victory   
                                                                35:38   Defeat   
                                                            3   37:27  Victory   
                                                                37:27   Defeat   
                                                            4   27:55   Defeat   
                                                                27:55  Victory   
                         2022-03-27 12.5    GEN(3) : DK(2)  1   38:20   Defeat   
                                               

## 리그/연도별 전체 데이터 크롤링

In [5]:
#리그/연도별 크롤링 후 csv 파일로 만들기
leagues = ['LCK', 'LPL', 'LCS', 'LEC']
years = [str(year) for year in range(2015, 2023)]

for league in leagues:
    for year in years:
        txt_list = get_txt_files('./', league, year)
        print(f'{league}_{year} txt 파일 개수 : {len(txt_list)}')

        df = crawling_team_data(txt_list)
        df.to_csv(f'./team csv files/{league}_{year}_team_data.csv')

LCK_2015 txt 파일 개수 : 3


LCK_2015_Summer_Regular season_match_links.txt: 100%|██████████| 90/90 [29:44<00:00, 19.83s/it]


총 소요 시간 : 32.57분
----------------------------------------------------------------------------------------------------
LCK_2016 txt 파일 개수 : 8


LCK_2016_Summer_Playoffs_match_links.txt: 100%|██████████| 4/4 [01:05<00:00, 16.42s/it]
LCK_2016_Summer_Promotion relegation_match_links.txt: 100%|██████████| 2/2 [00:43<00:00, 21.51s/it]
LCK_2016_Summer_Regular season_match_links.txt: 100%|██████████| 90/90 [30:28<00:00, 20.32s/it]


총 소요 시간 : 80.22분
----------------------------------------------------------------------------------------------------
LCK_2017 txt 파일 개수 : 7


...

총 소요 시간 : 39.73분
----------------------------------------------------------------------------------------------------
LEC_2021 txt 파일 개수 : 4


LEC_2021_Summer_Regular Season_match_links.txt: 100%|██████████| 90/90 [17:24<00:00, 11.61s/it]


총 소요 시간 : 40.52분
----------------------------------------------------------------------------------------------------
LEC_2022 txt 파일 개수 : 3


LEC_2022_Summer_Regular Season_match_links.txt: 100%|██████████| 75/75 [14:26<00:00, 11.56s/it]


총 소요 시간 : 34.55분
----------------------------------------------------------------------------------------------------
